In [405]:
import pandas as pd

DATA_PATH = "../../data/"

In [406]:
corpus_df = pd.read_csv(DATA_PATH + "basic_cleaned_corpus_.csv")

In [407]:
corpus_df["string_traces"] = corpus_df["string_traces"].apply(eval)
corpus_df["num_traces"] = corpus_df["string_traces"].apply(len)
corpus_df["unique_activities"] = corpus_df["unique_activities"].apply(eval)
corpus_df["avg_trace_length"] = corpus_df["string_traces"].apply(lambda x: sum([len(t) for t in x]) / len(x))
corpus_df["max_trace_length"] = corpus_df["string_traces"].apply(lambda x: max([len(t) for t in x]))
corpus_df["min_trace_length"] = corpus_df["string_traces"].apply(lambda x: min([len(t) for t in x]))
corpus_df["num_unique_activities"] = corpus_df["unique_activities"].apply(len)
corpus_df = corpus_df[corpus_df["num_unique_activities"] > 1]

In [395]:
corpus_df["num_traces"].min()

1

In [392]:
len(corpus_df)

15857

In [398]:
all_unique = set(tuple(x) for s in corpus_df["string_traces"].values for x in s)

In [399]:
len(all_unique)

163484

In [ ]:
model_df = pd.read_csv(DATA_PATH + "eval_data_ooo.csv")
#model_df = model_df[model_df["language"]=="en"]
#model_df = model_df.drop_duplicates(subset=['unique_activities'])
print(len(model_df))

In [420]:
import pickle
with open(DATA_PATH +'train_val_test.pkl', 'rb') as f:
    # Load the pickled data
    train_ids, val_ids, test_ids = pickle.load(f)

In [421]:
trace_df = pd.read_pickle(DATA_PATH + "eval/eval_train_data_traces_balanced.pkl")
pair_df = pd.read_pickle(DATA_PATH + "eval/eval_train_data_pairs_balanced.pkl")
prefix_df = pd.read_pickle(DATA_PATH + "eval/eval_train_prefix_data.pkl")
trace_df["id"] = trace_df["model_id"].astype(str) + "_" + trace_df["revision_id"].astype(str)
pair_df["id"] = pair_df["model_id"].astype(str) + "_" + pair_df["revision_id"].astype(str)
prefix_df["id"] = prefix_df["model_id"].astype(str) + "_" + prefix_df["revision_id"].astype(str)

In [433]:
len(trace_df[~trace_df["anomalous"]])

150301

In [422]:
train_df = trace_df[trace_df["id"].isin(train_ids)]
val_df = trace_df[trace_df["id"].isin(val_ids)]
test_df = trace_df[trace_df["id"].isin(test_ids)]
print("Traces", len(trace_df), len(train_df), len(val_df), len(test_df))

Traces 291251 227892 43609 19750


In [423]:
train_df = pair_df[pair_df["id"].isin(train_ids)]
val_df = pair_df[pair_df["id"].isin(val_ids)]
test_df = pair_df[pair_df["id"].isin(test_ids)]
print("Pairs", len(pair_df), len(train_df), len(val_df), len(test_df))

Pairs 316308 229402 56154 30752


In [424]:
train_df = prefix_df[prefix_df["id"].isin(train_ids)]
val_df = prefix_df[prefix_df["id"].isin(val_ids)]
test_df = prefix_df[prefix_df["id"].isin(test_ids)]
print("Prefixes", len(prefix_df), len(train_df), len(val_df), len(test_df))

Prefixes 1289081 1071529 166811 50741


In [ ]:
def get_prefixes(x):
    tr = x["trace"]
    return [tr[:i] for i in range(1, len(tr))] +[tr], [tr[i] for i in range(1, len(tr))] +  ['[END]']


def generate_prefix_data(df):
    df[["prefixes", "next"]] = df.apply(lambda x: pd.Series([*get_prefixes(x)]), axis=1)
    return df


def expand_df_regular_traces(df):
    per_trace_records = []
    for i, row in df.iterrows():
        traces = eval(row['string_traces'])
        labels = eval(row['labels'])
        for tix, trace in enumerate(traces):
            if len(trace) < 2:
                continue
            per_trace_records.append({
                'model_id': row['model_id'],
                'revision_id': row['revision_id'],
                'trace': traces[tix],
                'unique_activities': row['unique_activities']
            })

    df = pd.DataFrame(per_trace_records)
    return df


def expand_trace_df(df):
    per_trace_records = []
    for i, row in df.iterrows():
        for prefix, next_ in zip(row["prefixes"], row['next']):
            per_trace_records.append({
                'model_id': row['model_id'],
                'revision_id': row['revision_id'],
                'trace': row['trace'],
                'prefix': prefix,
                'next': next_,
                'unique_activities': row['unique_activities']
            })
    df = pd.DataFrame(per_trace_records)
    return df

In [ ]:
trace_df = expand_df_regular_traces(model_df)
trace_df

In [ ]:
trace_df = generate_prefix_data(trace_df)
trace_df

In [ ]:
prefix_df = expand_trace_df(trace_df)

In [ ]:
prefix_df

In [ ]:
# pickle the data
prefix_df.to_pickle(DATA_PATH + "eval_train_prefix_data.pkl")

In [ ]:
model_df["allowed"][5000]

In [ ]:
model_df["disallowed"][5000]

In [ ]:
# randomly split the dataframe so that one has 10% of the rows and the other 90%
from sklearn.model_selection import train_test_split

# Set test_size to 0.1 to get 10% of the rows in the test set
train_df, test_df = train_test_split(model_df, test_size=0.3,random_state=4)

# compute how many activities in 'unique_activities' in rows of the test_df appear in the rows that are in the train_df
all_activities_in_train = {a for l in train_df['unique_activities'].values for a in eval(l)}
print(len(all_activities_in_train))
all_activities_in_test = {a for l in test_df['unique_activities'].values for a in eval(l)}
print(len(all_activities_in_test))
print(len(all_activities_in_train.intersection(all_activities_in_test)))
test_df

In [ ]:
# next we generate dataframe records such that rach individual trace is represented in a row
def expand_df(df):    
    per_trace_records = []
    for i, row in df.iterrows():
        noisy_traces = eval(row['noisy'])
        labels = eval(row['labels'])
        for tix, trace in noisy_traces.items():
            per_trace_records.append({
                'model_id': row['model_id'],
                'revision_id': row['revision_id'],
                'pt': row['pt'],
                'trace': noisy_traces[tix],
                'label': labels[tix],
                'unique_activities': row['unique_activities']
            })
            
    df = pd.DataFrame(per_trace_records)
    return df

In [ ]:
trace_df = expand_df(model_df)
#train_df = expand_df(train_df)
#test_df = expand_df(test_df)

In [ ]:

#test_df.to_pickle(DATA_PATH + "eval_test_data_traces.pkl")
#train_df.to_pickle(DATA_PATH + "eval_train_data_traces.pkl")

In [ ]:
def expand_df_per_eventually_follows_pair(df):
    per_trace_records = []
    for i, row in df.iterrows():
        noisy_trace = row['trace']
        out_of_order_anomalies = [tuple(anomaly[1]) for anomaly in row['label'] if anomaly[0]==0]
        
        ef_pairs = set()
        for j in range(len(noisy_trace)-1):
            for k in range(j+1, len(noisy_trace)):
                ef_pairs.add((noisy_trace[j], noisy_trace[k]))
        for pair in ef_pairs:
            out_of_order = True if pair in out_of_order_anomalies else False
            per_trace_records.append({
                'model_id': row['model_id'],
                'revision_id': row['revision_id'],
                'pt': row['pt'],
                'trace': noisy_trace,
                'out_of_order': out_of_order,
                'unique_activities': row['unique_activities'],
                'eventually_follows': pair
            })
    df = pd.DataFrame(per_trace_records)
    return df
    

In [ ]:
def eval_(s):
    if type(s) == str:
        return eval(s)
    else:
        return s

import random
def expand_df_per_pair(df):
    per_trace_records = []
    for i, row in df.iterrows():
        ef_pairs = eval_(row['allowed'])
        not_allowed = eval_(row['disallowed'])
        random.seed(4)
        all_pairs = ef_pairs + not_allowed
        random.shuffle(all_pairs)
        for pair in all_pairs:
            out_of_order = True if pair in not_allowed else False
            per_trace_records.append({
                'model_id': row['model_id'],
                'revision_id': row['revision_id'],
                'pt': row['pt'],
                'out_of_order': out_of_order,
                'unique_activities': row['unique_activities'],
                'eventually_follows': pair
            })
    df = pd.DataFrame(per_trace_records)
    return df

In [ ]:
pair_df = expand_df_per_pair(model_df)
#train_df = expand_df_per_eventually_follows_pair(train_df)
#test_df = expand_df_per_eventually_follows_pair(test_df)

In [ ]:
pair_df['out_of_order'].value_counts()

In [ ]:
trace_df["anomalous"] = trace_df['label'].apply(lambda x: len(x)>0)
trace_df["anomalous"].value_counts()

In [ ]:
pair_df['eventually_follows'].nunique()

In [ ]:
trace_df.to_pickle(DATA_PATH + "eval_train_data_traces_balanced.pkl")

In [ ]:
pair_df.to_pickle(DATA_PATH + "eval_train_data_pairs_balanced.pkl")

In [ ]:
pair_df

In [ ]:
pair_df = pd.read_pickle(DATA_PATH + "eval_train_data_pairs.pkl")
# create a train test split such that the same model_id and revision_id are not in both the train and test set
pair_df["id"] = pair_df["model_id"].astype(str) + "_" + pair_df["revision_id"].astype(str)
model_ids = pair_df['id'].unique()
train_ids, test_ids = train_test_split(model_ids, test_size=0.5, random_state=4)
train_df = pair_df[pair_df['id'].isin(train_ids)]
test_df = pair_df[pair_df['id'].isin(test_ids)]


In [ ]:
train_df_pairs = pd.read_pickle(DATA_PATH + "eval_train_data_pairs.pkl")

In [ ]:
train_df_pairs

In [ ]:
train_df_pairs['out_of_order'].value_counts()

In [ ]:
import pandas as pd

In [ ]:
prefix_df = pd.read_pickle(DATA_PATH + "eval_train_prefix_data.pkl")

In [ ]:
prefix_df = prefix_df[~prefix_df["next"].str.contains("END")]

In [ ]:
prefix_df.head(10)

In [ ]:
pair_df = pd.read_pickle(DATA_PATH + "eval/eval_train_data_pairs_balanced.pkl")

In [ ]:
pair_df

In [346]:
model_df = pd.read_csv(DATA_PATH + "eval_data_ooo.csv")

In [347]:
import numpy as np
from sklearn.model_selection import train_test_split

def split_by_model(df, split_sizes: list[float] = [0.2, 0.1], random_state=4) -> tuple[
    pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    df["id"] = df["model_id"].astype(str) + "_" + df["revision_id"].astype(str)
    df["string_traces"] = df["string_traces"].apply(lambda x: eval(x))
    df["num_activities"] = df["unique_activities"].apply(lambda x: int(len(eval(x))) if isinstance(x, str) else int(len(x)))
    df["num_activities"] = df["num_activities"].apply(lambda x: 15 if x > 15 else x)

    #df["avg_trace_length"] = df["string_traces"].apply(lambda x: np.mean([len(t) for t in x]))
    print(df["num_activities"].value_counts())
    model_ids = df["id"]
    train_val_ids, test_ids = train_test_split(
        model_ids, test_size=split_sizes[-1], random_state=random_state, stratify=df["num_activities"]
    )
    train_ids, val_ids = train_test_split(
        train_val_ids, test_size=split_sizes[-2], random_state=random_state, stratify=df[df["id"].isin(train_val_ids)]["num_activities"]
    )
    train_df = df[df["id"].isin(train_ids)]
    val_df = df[df["id"].isin(val_ids)]
    test_df = df[df["id"].isin(test_ids)]

    train_ = set(tuple(t) for l in train_df.string_traces.values for t in l)
    seen_val_df = val_df[val_df["string_traces"].apply(lambda x: any(tuple(t) in train_ for t in x))]
    seen_test_df = test_df[test_df["string_traces"].apply(lambda x: any(tuple(t) in train_ for t in x))]
    val_df = val_df[~val_df["string_traces"].apply(lambda x: any(tuple(t) in train_ for t in x))]
    test_df = test_df[~test_df["string_traces"].apply(lambda x: any(tuple(t) in train_ for t in x))]

    train_df = pd.concat([train_df, seen_test_df], ignore_index=True)
    train_df = pd.concat([train_df, seen_val_df], ignore_index=True)

    return list(train_df["id"].values), list(val_df["id"].values), list(test_df["id"].values)

In [348]:
train_df, val_df, test_df = split_by_model(model_df)
print(train_df,val_df,test_df)

num_activities
4     3147
3     3030
2     2429
5     2282
6     1838
7     1188
8      970
9      457
10     314
11     109
12      52
1       45
13      20
14       7
15       5
Name: count, dtype: int64
['e8990a10f516495e89a1eabf9627b9e9_295ab34872b6424aad0b4633abb621a9', 'e89a9255406a491cab3bd827a043dd28_dcc8dbe04d4347cd88c16bb77f77c843', 'e89e0c0e06c440e2a006c32d1d25e3f7_9a249351fb0f49dca3760032b9fdaea2', 'e8a0fe270519408f9df49651977afc8e_390d8410e2774873ad1f502a776a859f', 'e8a31cb70a6546b187326a111db648fe_ae2e746b74834904bbb125067f4a4924', 'e8a4b883dcda4483960f3ff24f3bbeda_c126e55d6a9640de9b0c2fc156824c1f', 'e8a4e2d3a8bf46eb884e7a6518d4efb7_6f51d14d60544fe0ae8761b64b8c1e36', 'e8a997b7747d40e0b2c7e0c3716f25f8_2cbb341de8a2430db7d3d4c85331134a', 'e8af4e14cdf14367b85b270119a7a88a_f28c567af7ce4f6c8a7a96dd2a2d9802', 'e8b05b09010a4c7babb39613534526f0_73f4d8f3efb74fa39f6f9db05ffe7e78', 'e8b38c6a696548f39d75896cd473101e_806b163f766b4ae0a01b98f2a0b3e9af', 'e8b53b079e804658b3f9070f8b918f38_

In [354]:
import pickle
pickle.dump((train_df, val_df, test_df), open(DATA_PATH + "train_val_test.pkl", "wb"))

In [355]:
print(len(train_df), len(val_df), len(test_df))

11533 2803 1557


In [ ]:
# eliminate seen pairs from validation data
val_ = set(tuple(t) for l in val_df.string_traces.values for t in l)

In [ ]:
train_ = set(tuple(t) for l in train_df.string_traces.values for t in l)
intersect_val = train_.intersection(val_)
# eliminate seen pairs from test data
test_ = set(tuple(t) for l in test_df.string_traces.values for t in l)
intersect_test = train_.intersection(test_)

In [ ]:
len(test_)

In [ ]:
    train_traces = set(train_df["trace"].unique())
    # move seen samples from validation data train
    val_traces = set(val_df["trace"].unique())
    intersect = train_traces.intersection(val_traces)
    seen_v = val_df[val_df["trace"].isin(intersect)]
    val_df = val_df[~val_df["trace"].isin(intersect)]
    train_df = pd.concat([train_df, seen_v], ignore_index=True)
    # move seen samples from test data to train
    test_traces = set(test_df["trace"].unique())
    intersect = train_traces.intersection(test_traces)
    seen_t = test_df[test_df["trace"].isin(intersect)]
    test_df = test_df[~test_df["trace"].isin(intersect)]
    train_df = pd.concat([train_df, seen_t], ignore_index=True)
    train_df["trace"] = train_df["trace"].apply(lambda x: list(x))
    val_df["trace"] = val_df["trace"].apply(lambda x: list(x))
    test_df["trace"] = test_df["trace"].apply(lambda x: list(x))